## Текстовые функции и вложения в CatBoost

**Установите GPU в качестве аппаратного ускорителя**

    Прежде всего, вам нужно выбрать GPU в качестве аппаратного ускорителя. Для этого есть два простых шага:
    Шаг 1. Перейдите к **Runtime** меню и выберите пункт **Change runtime type**
    Шаг 2. Выбирать **GPU** в качестве аппаратного ускорителя.


In [ ]:
# !pip install catboost

In [10]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

import catboost
print(catboost.__version__)

0.24.2


## Подготовка данных

In [27]:
df = pd.read_csv('toxic_comments.csv')

In [28]:
df.shape

(159571, 2)

In [29]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [31]:
df['label'] = (df['toxic'] > 0).astype(int)
df.drop(['toxic'], axis=1, inplace=True)
df.head()

,text,label
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [32]:
df['label'].value_counts()

0    143346
1     16225
Name: label, dtype: int64

In [34]:
from catboost import Pool
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.8, random_state=0)
y_train, X_train = train_df['label'], train_df.drop(['label'], axis=1)
y_test, X_test = test_df['label'], test_df.drop(['label'], axis=1)

train_pool = Pool(data=X_train, label=y_train, text_features=['text'])
test_pool = Pool(data=X_test, label=y_test, text_features=['text'])

print('Train dataset shape: {}\n'.format(train_pool.shape))

Train dataset shape: (127656, 1)



In [38]:
from catboost import CatBoostClassifier
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.1,
        eval_metric='F1',
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=test_pool,
        verbose=100
    )

model = fit_model(train_pool, test_pool, task_type='GPU')

0:	learn: 0.6723927	test: 0.6973663	best: 0.6973663 (0)	total: 74.4ms	remaining: 1m 14s
100:	learn: 0.6857495	test: 0.7003308	best: 0.7005748 (98)	total: 7.71s	remaining: 1m 8s
200:	learn: 0.6988857	test: 0.7042352	best: 0.7049381 (149)	total: 14.8s	remaining: 58.8s
300:	learn: 0.7088843	test: 0.7066036	best: 0.7079398 (269)	total: 21.8s	remaining: 50.7s
400:	learn: 0.7144059	test: 0.7060266	best: 0.7079398 (269)	total: 28.5s	remaining: 42.6s
500:	learn: 0.7206808	test: 0.7059444	best: 0.7079398 (269)	total: 35.5s	remaining: 35.4s
600:	learn: 0.7257021	test: 0.7060271	best: 0.7084651 (522)	total: 42.3s	remaining: 28.1s
700:	learn: 0.7302824	test: 0.7063785	best: 0.7084651 (522)	total: 49.2s	remaining: 21s
800:	learn: 0.7338792	test: 0.7063994	best: 0.7084651 (522)	total: 56.2s	remaining: 14s
900:	learn: 0.7387252	test: 0.7082163	best: 0.7090207 (878)	total: 1m 3s	remaining: 6.92s
999:	learn: 0.7426678	test: 0.7080702	best: 0.7094962 (945)	total: 1m 9s	remaining: 0us
bestTest = 0.709496

## Как это работает

    1. Токенизация Текста
    2. Создание Словаря
    3. Расчет Характеристик

## Токенизация Текста

Обычно мы получаем наш текст в виде последовательности символов Юникода. Таким образом, если задача не является классификацией ДНК, нам не нужна такая детализация, более того, нам нужно извлечь более сложные сущности, например слова. Процесс извлечения токенов-слов, цифр, знаков препинания или специальных символов, которые определяют эмодзи из последовательности, называется **токенизацией**

Токенизация-это первая часть предварительной обработки текста в CatBoost и выполняется как простое разбиение последовательности на строковый шаблон (например, пробел)

In [ ]:
text_small = [
    "Cats are so cute :)",
    "Mouse scare...",
    "The cat defeated the mouse",
    "Cute: Mice gather an army!",
    "Army of mice defeated the cat :(",
    "Cat offers peace",
    "Cat is scared :(",
    "Cat and mouse live in peace :)"
]

target_small = [1, 0, 1, 1, 0, 1, 0, 1]

In [ ]:
from catboost.text_processing import Tokenizer

simple_tokenizer = Tokenizer()

def tokenize_texts(texts):
    return [simple_tokenizer.tokenize(text) for text in texts]

simple_tokenized_text = tokenize_texts(text_small)
simple_tokenized_text

### Дополнительная предварительная обработка

Давайте подробнее рассмотрим результат токенизации небольшого текстового примера-токены содержат много ошибок:

1. Они склеены пунктуацией 'Cute:', 'army!', 'skare...'.
2. Слово 'Cat' and 'cat', 'Mice' и 'mice' кажется, они имеют одно и то же значение, Возможно, это должны быть одни и те же символы.
3. Та же проблема и с токенами 'are'/'is' -- это флективные формы одного и того же знака 'be'.
    
    **Пунктуационная обработка** , **строчной**, и **лемматизация** процессы помогают преодолеть эти проблемы.

### Обработка знаков препинания и строчные буквы

In [ ]:
tokenizer = Tokenizer(
    lowercasing=True,
    separator_type='BySense',
    token_types=['Word', 'Number']
)

tokenized_text = [tokenizer.tokenize(text) for text in text_small]
tokenized_text

### Удаление стоп-слов

**Стоп - слова** -слова, которые считаются неинформативными в этой задаче, например функциональные слова, такие как, is, at, which, on. Обычно стоп-слова удаляются во время предварительной обработки текста, чтобы уменьшить объем информации, которая рассматривается для дальнейших алгоритмов. Стоп-слова собираются вручную (в виде словаря) или автоматически, например, беря наиболее частые слова

In [ ]:
stop_words = set(('be', 'is', 'are', 'the', 'an', 'of', 'and', 'in'))

def filter_stop_words(tokens):
    return list(filter(lambda x: x not in stop_words, tokens))
    
tokenized_text_no_stop = [filter_stop_words(tokens) for tokens in tokenized_text]
tokenized_text_no_stop

### лемматизация

Лемма (Википедия) - это каноническая форма, словарная форма или форма цитирования набора слов.
Например, Лемма "go" представляет собой формы "go", "goes", "going", "went", and "gone". Процесс преобразования слова в его лемму называется **лемматизация**.

In [ ]:
import nltk
nltk_data_path = os.path.join(os.path.dirname(nltk.__file__), 'nltk_data')
nltk.data.path.append(nltk_data_path)
nltk.download('wordnet', nltk_data_path)

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_tokens_nltk(tokens):
    return list(map(lambda t: lemmatizer.lemmatize(t), tokens))

In [ ]:
text_small_lemmatized_nltk = [lemmatize_tokens_nltk(tokens) for tokens in tokenized_text_no_stop]
text_small_lemmatized_nltk

Теперь слова с одинаковым значением представлены одним и тем же маркером, лексемы не склеиваются с пунктуацией.

Будьте осторожны. Вы должны проверить это для своей собственной задачи:
Действительно ли необходимо удалять знаки препинания, строчные предложения или выполнять лемматизацию и/или токенизацию слов?

### Давайте проверим точность с помощью новой предварительной обработки текста
Поскольку CatBoost не выполняет интервальную пунктуацию, строчные буквы и лемматизацию, нам нужно предварительно обработать текст вручную, а затем передать его алгоритму обучения.

Поскольку естественными текстовыми признаками являются только конспект и обзор, мы будем предварительно обрабатывать только их.

In [ ]:
%%time

def preprocess_data(X):
    X_preprocessed = X.copy()
    X_preprocessed['review'] = X['review'].apply(lambda x: ' '.join(lemmatize_tokens_nltk(tokenizer.tokenize(x))))
    return X_preprocessed

X_preprocessed_train = preprocess_data(X_train)
X_preprocessed_test = preprocess_data(X_test)

train_processed_pool = Pool(
    X_preprocessed_train, y_train, 
    text_features=['review'],
)

test_processed_pool = Pool(
    X_preprocessed_test, y_test, 
    text_features=['review'],
)

In [ ]:
model_on_processed_data = fit_model(train_processed_pool, test_processed_pool, task_type='GPU')

In [ ]:
def print_score_diff(first_model, second_model):
    first_accuracy = first_model.best_score_['validation']['AUC']
    second_accuracy = second_model.best_score_['validation']['AUC']

    gap = (second_accuracy - first_accuracy) / first_accuracy * 100

    print('{} vs {} ({:+.2f}%)'.format(first_accuracy, second_accuracy, gap))
    
print_score_diff(model, model_on_processed_data)

## Создание Словаря

После первого этапа, предварительной обработки текста и токенизации, начинается второй этап. Второй этап использует подготовленный текст для выбора набора единиц измерения, которые будут использоваться для построения новых числовых признаков.

Набор выбранных единиц называется словарем. Он может содержать слова, биграммы слов или символьные n-граммы.

In [ ]:
from catboost.text_processing import Dictionary

In [ ]:
text_small_lemmatized_nltk

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=10)

dictionary.fit(text_small_lemmatized_nltk);
#dictionary.fit(text_small, tokenizer)

In [ ]:
dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
dictionary.apply([text_small_lemmatized_nltk[0]])

## Расчет Характеристик

### Преобразование в векторы фиксированного размера

Большинство классических алгоритмов ML вычисляют и выполняют предсказания на фиксированном числе объектов $F$.<br>
Это означает, что набор обучения $X= (x_i) $ содержит векторы $x_i = (a_0, a_1, ..., a_F)$ где  $F$ константа.    

Так как текстовый объект $x$ это не вектор фиксированной длины, нам нужно выполнить предварительную обработку исходного набора $D$.<br>
Одним из самых простых методов кодирования текста в вектор является **Мешок слов (BoW)**.

### Алгоритм мешка слов

Алгоритм принимает в себя словарь и текст.<br>
Во время работы алгоритма текст $x = (a_0, a_1, ..., a_k)$ преобразовано в вектор $\\tilde x = (b_0, b_1, ..., b_F)$,<br> где  $b_i$ это 0/1 (в зависимости от того, есть ли слово с id=$i$ из словаря в текст $x$).

In [ ]:
X_proc_train_small, y_train_small = X_preprocessed_train[:1000]['review'].to_list(), y_train[:1000]
X_proc_train_small = list(map(simple_tokenizer.tokenize, X_proc_train_small))
X_proc_test_small, y_test_small = X_preprocessed_test[:1000]['review'].to_list(), y_test[:1000]
X_proc_test_small = list(map(simple_tokenizer.tokenize, X_proc_test_small))

dictionary = Dictionary(max_dictionary_size=100)
dictionary.fit(X_proc_train_small);

In [ ]:
def bag_of_words(tokenized_text, dictionary):
    features = np.zeros((len(tokenized_text), dictionary.size))
    for i, tokenized_sentence in enumerate(tokenized_text):
        indices = np.array(dictionary.apply([tokenized_sentence])[0])
        if len(indices) > 0:
            features[i, indices] = 1
    return features

X_bow_train_small = bag_of_words(X_proc_train_small, dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, dictionary)
X_bow_train_small.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.metrics import roc_auc_score

def fit_linear_model(X, y):
    model = LogisticRegression()
    model.fit(X, y)
    return model

def evaluate_model_auc(model, X, y):
    y_pred = model.predict_proba(X)[:,1]
    metric = roc_auc_score(y, y_pred)
    print('AUC: ' + str(metric))

In [ ]:
def evaluate_models(X_train, y_train, X_test, y_test):
    linear_model = fit_linear_model(X_train, y_train)
        
    print('Linear model')
    evaluate_model_auc(linear_model, X_test, y_test)
    print('Comparing to constant prediction')
    auc_constant_prediction = roc_auc_score(y_test, np.ones(shape=(len(y_test), 1)) * 0.5)
    print('AUC: ' + str(auc_constant_prediction))
    
evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

In [ ]:
unigram_dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=1000)
unigram_dictionary.fit(X_proc_train_small)

X_bow_train_small = bag_of_words(X_proc_train_small, unigram_dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, unigram_dictionary)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

### Глядя на последовательности букв / слов

Давайте рассмотрим пример: тексты "кошка победила мышь" и " армия мышей победила кошку:('<br>
Упрощая его, мы имеем три лексемы в каждом предложении "кошка побеждает мышь" и "мышь побеждает кошку'.<br>
После применения лука мы получаем два равных вектора с противоположным значением:

| cat | mouse | defeat |
|-----|-------|--------|
| 1   | 1     | 1      |
| 1   | 1     | 1      |

Как их отличить?
Давайте добавим последовательности слов в виде отдельных лексем в наш словарь:

| cat | mouse | defeat | cat_defeat | mouse_defeat | defeat_cat | defeat_mouse |
|-----|-------|--------|------------|--------------|------------|--------------|
| 1   | 1     | 1      | 1          | 0            | 0          | 1            |
| 1   | 1     | 1      | 0          | 1            | 1          | 0            |

**N-gram** это непрерывная последовательность $n$ элементов из заданного образца текста или речи (Wikipedia).<br>
В приведенном выше примере Bi-gram (Bigram)  = 2 слова

Nграммы помогают добавить в векторы больше информации о структуре текста, более того, существуют n-граммы, не имеющие значения в разделении, например, 'Mickey Mouse company'.

In [ ]:
dictionary = Dictionary(occurence_lower_bound=0, gram_order=2)
dictionary.fit(text_small_lemmatized_nltk)

dictionary.save('dictionary.tsv')
!cat dictionary.tsv

In [ ]:
bigram_dictionary = Dictionary(occurence_lower_bound=0, max_dictionary_size=5000, gram_order=2)
bigram_dictionary.fit(X_proc_train_small)

X_bow_train_small = bag_of_words(X_proc_train_small, bigram_dictionary)
X_bow_test_small = bag_of_words(X_proc_test_small, bigram_dictionary)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

### Unigram + Bigram

In [ ]:
X_bow_train_small = np.concatenate((
    bag_of_words(X_proc_train_small, unigram_dictionary),
    bag_of_words(X_proc_train_small, bigram_dictionary)
), axis=1)
X_bow_test_small = np.concatenate((
    bag_of_words(X_proc_test_small, unigram_dictionary),
    bag_of_words(X_proc_test_small, bigram_dictionary)
), axis=1)
print(X_bow_train_small.shape)

evaluate_models(X_bow_train_small, y_train_small, X_bow_test_small, y_test_small)

## CatBoost Configuration

Имя параметра:

1. **Text Tokenization** - `tokenizers`
2. **Dictionary Creation** - `dictionaries`
3. **Feature Calculation** - `feature_calcers`

\* Более сложная конфигурация с `text_processing` параметр

### `tokenizers`

Tokenizers used to preprocess Text type feature columns before creating the dictionary.

[Documentation](https://catboost.ai/docs/references/tokenizer_options.html).

In [ ]:
tokenizers = [{
    'tokenizerId': 'Space',
    'delimiter': ' ',
    'separator_type': 'ByDelimiter',
},{
    'tokenizerId': 'Sense',
    'separator_type': 'BySense',
}]

### `dictionaries`

Dictionaries used to preprocess Text type feature columns.

[Documentation](https://catboost.ai/docs/references/dictionaries_options.html).

In [ ]:
dictionaries = [{
    'dictionaryId': 'Unigram',
    'max_dictionary_size': '50000',
    'gram_count': '1',
},{
    'dictionaryId': 'Bigram',
    'max_dictionary_size': '50000',
    'gram_count': '2',
},{
    'dictionaryId': 'Trigram',
    'token_level_type': 'Letter',
    'max_dictionary_size': '50000',
    'gram_count': '3',
}]

### `feature_calcers`

Калькуляторы объектов используются для расчета новых объектов на основе предварительно обработанных столбцов объектов текстового типа.

1. **`BoW`**<br>
Мешок слов: 0/1 features (образец текста имеет или не имеет token_id).<br>
Количество произведенных числовые характеристики = размер словаря.<br>
параметр: `top_tokens_count` - максимальное количество токенов, которые будут использоваться для векторизации в мешке слов, наиболее частые $n$ жетоны принимаются (**сильно влияет как на использование CPU ang GPU RAM**).

2. **`NaiveBayes`**<br>
NaiveBayes: [Полиномиальное упрощенного алгоритма Байеса](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) модель. Добавлено столько же новых функций, сколько и классов. Эта функция вычисляется по аналогии со счетчиками в CatBoost путем перестановки ([оценка и показатели CTR](https://catboost.ai/docs/concepts/algorithm-main-stages_cat-to-numberic.html)). Другими словами, производится случайная перестановка, а затем мы идем сверху вниз по набору данных и вычисляем вероятность его принадлежности к этому классу для каждого объекта.

3. **`BM25`**<br>
[BM25](https://en.wikipedia.org/wiki/Okapi_BM25). Добавлено столько же новых функций, сколько и классов. Идея та же, что и в наивном Байесе, но для каждого класса мы вычисляем не условную вероятность, а определенную релевантность, что аналогично tf-idf, где лексемы вместо слов и классы вместо документов (точнее, объединение всех текстов этого класса). Только множитель tf в BM25 заменяется другим множителем, что дает преимущество классам, содержащим редкие токены.

In [ ]:
feature_calcers = [
    'BoW:top_tokens_count=1000',
    'NaiveBayes',
    'BM25',
]

### `text_processing`

In [ ]:
text_processing = {
    "tokenizers" : [{
        "tokenizer_id" : "Space",
        "separator_type" : "ByDelimiter",
        "delimiter" : " "
    }],

    "dictionaries" : [{
        "dictionary_id" : "BiGram",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "2"
    }, {
        "dictionary_id" : "Word",
        "max_dictionary_size" : "50000",
        "occurrence_lower_bound" : "3",
        "gram_order" : "1"
    }],

    "feature_processing" : {
        "default" : [{
            "dictionaries_names" : ["BiGram", "Word"],
            "feature_calcers" : ["BoW"],
            "tokenizers_names" : ["Space"]
        }, {
            "dictionaries_names" : ["Word"],
            "feature_calcers" : ["NaiveBayes"],
            "tokenizers_names" : ["Space"]
        }],
    }
}

## Резюме: текстовые функции в CatBoost

### Алгоритм:
1. Входной текст загружается в виде обычного столбца. ``text_column: [string]``.
2. Каждый образец текста маркируется с помощью разбиения на пробелы. ``tokenized_column: [[string]]``.
3. Оценка словаря.
4. Каждая строка в маркированном столбце преобразуется в token_id из словаря. ``text: [[token_id]]``.
5. В зависимости от параметров CatBoost производит функции на основе результирующего текстового столбца: Bag of words, Multinomial naive bayes или Bm25.
6. Вычисленные объекты float передаются в обычный алгоритм обучения CatBoost.


# Embeddings In CatBoost

### Получить Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer
big_model = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')
X_embed_train = big_model.encode(X_train['review'].to_list())
X_embed_test = big_model.encode(X_test['review'].to_list())

#!wget https://transfersh.com/HDHxy/embedded_train.npy -O embedded_train.npy
#X_embed_train = np.load('embedded_train.npy')

#!wget https://transfersh.com/whOm3/embedded_test.npy -O embedded_test.npy
#X_embed_test = np.load('embedded_test.npy')

### Experiments

In [ ]:
X_embed_first_train_small, y_first_train_small = X_embed_train[:5000], y_train[:5000]
X_embed_second_train_small, y_second_train_small = X_embed_train[5000:10000], y_train[5000:10000]
X_embed_test_small, y_test_small = X_embed_test[:5000], y_test[:5000]

#### Чистые embeddings

In [ ]:
evaluate_models(X_embed_second_train_small, y_second_train_small, X_embed_test_small, y_test_small)

#### линейный дискриминантный анализ

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(solver='svd')
lda.fit(X_embed_first_train_small, y_first_train_small)

X_lda_train_small = lda.transform(X_embed_second_train_small)
X_lda_test_small = lda.transform(X_embed_test_small)
print(X_lda_train_small.shape)
evaluate_models(X_lda_train_small, y_second_train_small, X_lda_test_small, y_test_small)

## Embeddings in CatBoost

In [ ]:
import csv
with open('train_embed_text.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t', quotechar='"')
    for y, text, row in zip(y_train, X_preprocessed_train['review'].to_list(), X_embed_train):
        writer.writerow((str(y), text, ';'.join(map(str, row))))

with open('test_embed_text.tsv', 'w') as f:
    writer = csv.writer(f, delimiter='\t', quotechar='"')
    for y, text, row in zip(y_test, X_preprocessed_test['review'].to_list(), X_embed_test):
        writer.writerow((str(y), text, ';'.join(map(str, row))))
        
with open('pool_text.cd', 'w') as f:
    f.write(
        '0\tLabel\n'\
        '1\tText\n'\
        '2\tNumVector'
    )

In [ ]:
from catboost import Pool
train_embed_pool = Pool('train_embed_text.tsv', column_description='pool_text.cd')
test_embed_pool = Pool('test_embed_text.tsv', column_description='pool_text.cd')

In [ ]:
model_text_embeddings = fit_model(train_embed_pool, test_embed_pool)

In [ ]:
print_score_diff(model, model_text_embeddings)